In [2]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
HOME = expanduser("~")
import os, sys
import SimpleITK as sitk
from PIL import Image, ImageOps
from scipy.ndimage import rotate
from tqdm import tqdm
from IPython.display import clear_output
REPO_DIR = '/home/eddyod/programming/pipeline_utility'
sys.path.append(REPO_DIR)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
stack = 'DK39'
sqlController = SqlController()
fileLocationManager = FileLocationManager(stack)
image_name_list = sqlController.get_image_list(stack, 'destination')

#%matplotlib inline

Connecting dklab@localhost:3306


In [14]:
BADS = ['DK39_ID_0001_slide001_S1_C1.tif', 'DK39_ID_0007_slide001_S2_C1.tif', 'DK39_ID_0010_slide001_S3_C1.tif',
       'DK39_ID_0013_slide002_S1_C1.tif', 'DK39_ID_0073_slide007_S1_C1.tif', 'DK39_ID_0823_slide071_S3_C1.tif',
       'DK39_ID_0004_slide001_S4_C1.tif',
 'DK39_ID_0016_slide002_S2_C1.tif',
 'DK39_ID_0019_slide002_S3_C1.tif',
 'DK39_ID_0022_slide002_S4_C1.tif',
 'DK39_ID_0034_slide003_S1_C1.tif',]
files = [i for i in image_name_list if '_C1' in i and i not in BADS] 
files = files[100:300]

In [ ]:
len(files)

In [12]:
DIR = os.path.join(HOME, 'programming', 'dk39')
#DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39'
ORIENTED = os.path.join(DIR, 'preps', 'oriented')
PREALIGNED = os.path.join(DIR, 'preps', 'prealigned')
ALIGNED = os.path.join(DIR, 'preps', 'aligned')
NORMALIZED = os.path.join(DIR, 'preps', 'normalized')
POSTALIGNED = os.path.join(DIR, 'preps', 'postaligned')
PADDED = os.path.join(DIR, 'preps', 'padded')
THUMBNAIL = os.path.join(DIR, 'preps', 'thumbnail')
INPUT = ORIENTED
#BADS = ['DK39_ID_0001_slide001_S1_C1.tif', 'DK39_ID_0007_slide001_S2_C1.tif', 'DK39_ID_0010_slide001_S3_C1.tif']
#INPUTS = INPUTS[80:120]
COLOR = 2**16

In [9]:
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()
    
# Callback invoked when the IterationEvent happens, update our data and display new figure.
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))

def create_resample(fixed_image, moving_image):
    
    transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image,moving_image.GetPixelID()), 
                                                      moving_image, 
                                                      sitk.Euler2DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
    
    dimension = moving_image.GetDimension()
    reference_physical_size = np.zeros(dimension)
    reference_origin = np.zeros(dimension)
    reference_direction = np.identity(dimension).flatten()
    #transform = sitk.AffineTransform(dimension)
    #transform.SetMatrix(moving_image.GetDirection())
    #transform.SetTranslation(np.array(moving_image.GetOrigin()) - reference_origin)
    reference_physical_size[:] = [(sz-1)*spc if sz*spc>mx  else mx for sz,spc,mx 
                                  in zip(moving_image.GetSize(), moving_image.GetSpacing(), 
                                         reference_physical_size)]
    resample = sitk.ResampleImageFilter()
    
    # Create the reference image with a zero origin, identity direction cosine matrix and dimension     
    #reference_size = [1738]*dimension # Arbitrary sizes, smallest size that yields desired results. 
    reference_size = [1738, 998]
    reference_spacing = [ phys_sz/(sz-1) for sz,phys_sz in zip(reference_size, reference_physical_size) ]
    
    fixed_image = sitk.Image(reference_size, fixed_image.GetPixelIDValue())
    fixed_image.SetOrigin(reference_origin)
    fixed_image.SetSpacing(reference_spacing)
    fixed_image.SetDirection(reference_direction)
    
    resample.SetReferenceImage(fixed_image)

    # SimpleITK supports several interpolation options, we go with the simplest that gives reasonable results.     
    resample.SetInterpolator(sitk.sitkLinear)  
    resample.SetTransform(transform)
    #sitk.WriteImage(resample.Execute(moving_image), outputfile_prefix+'.mha')
    img = resample.Execute(moving_image)
    #img = sitk.Resample(img, reference_image, centered_transform, sitk.sitkLinear, 0.0)

    #print(transform)
    return img

def simple_resample(fixed_image, moving_image):
    #parameterMap = sitk.GetDefaultParameterMap('affine')
    #elastixImageFilter = sitk.ElastixImageFilter()
    #elastixImageFilter.SetParameterMap(parameterMap)
    #elastixImageFilter.Execute()
    #return  sitk.ElastixImageFilter(fixed_image, moving_image)
    
    #parameterMap = sitk.GetDefaultParameterMap("rigid")
    #parameterMap["Transform"] = ["AffineTransform"]

    #elastixImageFilter = sitk.ElastixImageFilter()
    #elastixImageFilter.SetFixedImage(fixed_image)
    #elastixImageFilter.SetMovingImage(moving_image)
    #elastixImageFilter.SetParameterMap(parameterMap)
    #elastixImageFilter.Execute()
    #return elastixImageFilter.GetResultImage()
    return  sitk.Elastix(fixed_image, moving_image)



In [15]:
img_inputs = []
img_outputs = []
file_inputs = []
movingFile = files[0]
file_inputs.append(movingFile)
movingPath = os.path.join(INPUT, movingFile)
fixedPath = movingPath
fixed_image =  sitk.ReadImage(fixedPath, sitk.sitkFloat32)
moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
fixed_image = simple_resample(fixed_image, moving_image)
#fixed_image = simple_resample(fixed_image, moving_image)
img_outputs.append(sitk.GetArrayFromImage(fixed_image))

orient = io.imread(movingPath)
img_inputs.append(orient)


for file in tqdm(files):
    movingFile = file
    file_inputs.append(movingFile)
    movingPath = os.path.join(INPUT, movingFile)
    moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
    fixed_image = simple_resample(fixed_image, moving_image)
    img_outputs.append(sitk.GetArrayFromImage(fixed_image))
    orient = io.imread(movingPath)
    img_inputs.append(orient)


OUTPUT = PADDED
dels = os.listdir(OUTPUT)
for d in dels:
    os.unlink(os.path.join(OUTPUT, d))

for i, img in enumerate(img_outputs):   
    
    filename = '{}.tif'.format(str(i).zfill(4))
    outfile = os.path.join(OUTPUT, filename)
    flat = img.flatten()
    fmax = int(flat.max())
    fmin = int(flat.min())
    flat = flat + abs(fmin)
    img = np.reshape(flat, img.shape)    
    io.imsave(outfile, img.astype('uint16'), check_contrast=False)
    img = None
print('done')

100%|██████████| 200/200 [16:28<00:00,  4.94s/it]


done


In [ ]:
start, finish = 0,10
for name,file in zip(file_inputs[start:finish], img_outputs[start:finish]):
    plt.style.use('classic')
    plt.figure()
    plt.title('{} {}'.format(name, file.shape))
    plt.imshow(file, cmap='gray')
    plt.show()

In [ ]:
data = []
file_inputs = []
for movingFile in files:
    file_inputs.append(movingFile)
    movingPath = os.path.join(INPUT, movingFile)
    data.append(sitk.ReadImage(movingPath))

In [ ]:

def create_reg(fixed_image, moving_image):
    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler2DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)

    moving_resampled = sitk.Resample(moving_image, fixed_image, initial_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

    
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(learningRate=0.5, numberOfIterations=150, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.            
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    # Connect all of the observers so that we can perform plotting during registration.
    registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
    registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
    registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations) 
    registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                   sitk.Cast(moving_image, sitk.sitkFloat32))
    return sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())